# Run Train of Bubble-Agent (w/ IQN)

- Team: TToBoT
- Member: { Sejun, Steve, Victor } @kaist

## Objective

- run training simultaneously w/ notebook


In [1]:
import os, sys, gin

# use parent folder as shared lib path..
if "../" not in sys.path:
    sys.path.append("../")

# major libraries
import gin.tf
import seaborn as sns
import matplotlib.pyplot as plt
from absl import flags
import numpy as np
import tensorflow as tf

# show tf version.
print('! tf.ver = {}'.format(tf.__version__))

! tf.ver = 1.15.2


In [2]:
# Globals
# BASE_PATH = './!experimental_results_bubble/run3'

# let Dopamine .py files to be imported as modules in Jupiter notebook
module_path = os.path.abspath(os.path.join('../dopamine'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(module_path)

/tf/dopamine


In [3]:
# try to load `Dopamine` libraries
import bubble
from dopamine.colab import utils as colab_utils

## Train Bubble w/ IQN

In [4]:
# @title Load the configuration for IQN.

# DQN_PATH = os.path.join(BASE_PATH, 'rainbow')
# Modified from dopamine/agents/implicit_quantile/config/implicit_quantile.gin

# CONFIG FOR IQN (see @bubble/iqn_nature.gin)
gin_config = '''
# run train for bubble agent
# - NOTE: customized for bubble w/ IQN
# - origin from `dopamine/agents/implicit_quantile/configs/implicit_quantile.gin`
#
# [RUN TRAIN]
# $ python -um dopamine.discrete_domains.train --base_dir=/tmp/bubble_iqn1 --gin_files='bubble/iqn_bubble.gin' --gin_bindings='RainbowAgent.tf_device="/cpu:*"'

# Hyperparameters follow Dabney et al. (2018), but we modify as necessary to
# match those used in Rainbow (Hessel et al., 2018), to ensure apples-to-apples
# comparison.
import dopamine.agents.implicit_quantile.implicit_quantile_agent
import dopamine.agents.rainbow.rainbow_agent
import dopamine.discrete_domains.atari_lib
import dopamine.discrete_domains.run_experiment
import dopamine.replay_memory.prioritized_replay_buffer
import gin.tf.external_configurables

# agent for bubble
import bubble.retro_lib
import bubble.bubble_agent
retro_lib.create_retro_environment.game_name = 'BubbleBobble'
retro_lib.create_retro_environment.level = 1
Runner.create_environment_fn = @retro_lib.create_retro_environment
create_agent.agent_name = 'implicit_quantile'
RetroPreprocessing.wall_offset = 0

ImplicitQuantileAgent.kappa = 1.0
ImplicitQuantileAgent.num_tau_samples = 64
ImplicitQuantileAgent.num_tau_prime_samples = 64
ImplicitQuantileAgent.num_quantile_samples = 32
RainbowAgent.gamma = 0.99
RainbowAgent.update_horizon = 3
RainbowAgent.min_replay_history = 20000 # agent steps
RainbowAgent.update_period = 4
RainbowAgent.target_update_period = 8000 # agent steps
RainbowAgent.epsilon_train = 0.01
RainbowAgent.epsilon_eval = 0.001
RainbowAgent.epsilon_decay_period = 250000  # agent steps
# IQN currently does not support prioritized replay.
RainbowAgent.replay_scheme = 'uniform'
RainbowAgent.tf_device = '/gpu:0'  # '/cpu:*' use for non-GPU version
RainbowAgent.optimizer = @tf.train.AdamOptimizer()

tf.train.AdamOptimizer.learning_rate = 0.00005
tf.train.AdamOptimizer.epsilon = 0.0003125

# atari_lib.create_atari_environment.game_name = 'Pong'
# Sticky actions with probability 0.25, as suggested by (Machado et al., 2017).
# atari_lib.create_atari_environment.sticky_actions = True
# create_agent.agent_name = 'implicit_quantile'
Runner.num_iterations = 200
Runner.training_steps = 250000
Runner.evaluation_steps = 125000
Runner.max_steps_per_episode = 27000

WrappedPrioritizedReplayBuffer.replay_capacity = 1000000
WrappedPrioritizedReplayBuffer.batch_size = 32
'''

# parse this config
gin.parse_config(gin_config, skip_unknown=False)


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
# Train DQN on Cartpole
#dqn_runner = create_runner(DQN_PATH, schedule='continuous_train')
#print('\n\n\nStart Training...\n\n\n')
#dqn_runner.run_experiment()
#print('\n\n\nDone training\n\n\n')
#dqn4 (5/28) - reward := -0.01 + 1*K - 3*D + log(S,100) + 5*L
#iqn7 (6/04) - final reward
DQN_PATH = '/tmp/bubble_iqn7'    # 5 -> wall_offset = 0

# import main run()
from dopamine.discrete_domains import run_experiment

# config main file
gin_files = []
# bindings.....
gin_bindings = ['Runner.evaluation_steps=0']

# # code from train.main()
# tf.logging.set_verbosity(tf.logging.INFO)
# run_experiment.load_gin_configs(gin_files, gin_bindings)
# runner = run_experiment.create_runner(DQN_PATH)

# # start run
# runner.run_experiment()


## Thread for updating status

In [8]:
# Thread for update canvas
import threading, time
def get_ioloop():
    import IPython, zmq
    ipython = IPython.get_ipython()
    if ipython and hasattr(ipython, 'kernel'):
        return zmq.eventloop.ioloop.IOLoop.instance()
# The IOloop is shared
ioloop = get_ioloop()
# Main Thread
class MyThread(threading.Thread):
    '''Thread for drawing into canvas in live'''
    def __init__(self, sleep = 0.5, name = 'my'):
        super().__init__()
        self._quit = threading.Event()
        self.sleep = 0.5
        self.name = name
        self.start()    
    def run(self):
        while not self._quit.isSet():
            def update_progress():
                if self._quit.isSet():
                    return
                self.display()
            time.sleep(self.sleep)
            ioloop.add_callback(update_progress)
        print("! T[{}].Quit()".format(self.name))
    def quit(self):
        self._quit.set()
    def display(self):
        pass

# display basic 
from ipycanvas import Canvas
canvas = Canvas(width=640, height=480)
if canvas:
    canvas.stroke_text('hello canvas! -------------', 0, 10)
# show canvas in here.
canvas

Canvas(height=480, width=640)

In [9]:
# Helper for Canvas
#canvas.fill_style = 'green'
#canvas.fill_rect(25, 25, 100, 100)
#canvas.clear_rect(45, 45, 60, 60)
def drawPlot2Canvas(fig = None, x=0, y=0):
    '''draw current plt to canvas at (x,y)'''
    fig = plt.gcf() if fig is None else fig
    plt.close()          # not to update on screen.
    fig.canvas.draw()    # draw fig to canvas
    arr = np.array(fig.canvas.renderer._renderer)
    print('! arr = {}'.format(np.shape(arr)))
    h, w, d = np.shape(arr)
    print('! w,h,d = {}'.format(w))
    cv = Canvas(width=w, height=h)
    cv.put_image_data(arr, 0, 0)
    cv.stroke_rect(x, y, x+w-1, y+h-1)
    canvas.clear_rect(x,y,x+w,y+h)
    canvas.draw_image(cv, x, y)
def drawText2Canvas(txt='msg!', x=10, y=10):
    w,h,o = 200,10,10
    #canvas.fill_style = 'green'
    #canvas.fill_rect(x, y-o, x+w, y+h-o)
    canvas.clear_rect(x, y-o, x+w, y+h-o)
    canvas.stroke_text(txt, x, y)
# draw plot....
fig = plt.figure(1)
plt.plot([[1,3],[3,3],[7,1]])
# draw plot-to-canvas
drawPlot2Canvas(fig, x=0)
drawText2Canvas('hello world')

! arr = (288, 432, 4)
! w,h,d = 432


In [ ]:
#drawText2Canvas('......................')

### support Multi-Processing

In [10]:
from multiprocessing import Process, Queue
# process list
proc_list = []
proc_queue = None

# train function
def processTrain(name = 'train', Q = None):
    global gin_files, gin_bindings, DQN_PATH
    from dopamine.discrete_domains import run_experiment
    Q.put('init!') if Q else None
    tf.logging.set_verbosity(tf.logging.INFO)
    run_experiment.load_gin_configs(gin_files, gin_bindings)
    runner = run_experiment.create_runner(DQN_PATH)
    # access to env
    env = runner._environment
    o = env.reset()
    Q.put('! o({}) = {}'.format(type(o), o[0:10,0,]))
    Q.put('start!') if Q else None
    runner.run_experiment()
    Q.put('! P[{}].stop()'.format(name))

# train thread
def startProcessTrain(target = None):
    global proc_queue, proc_list
    target = target if target is not None else processTrain
    proc_queue = Queue() if proc_queue is None else proc_queue
    proc = Process(target = target, args = ('T0', proc_queue))
    proc_list.append(proc)
    proc.start()
    return proc

# stop(or kill) processes
def stopProcess():
    global proc_list
    for proc in proc_list:
        proc.terminate()
        proc.join()

In [ ]:
# trainer = startProcessTrain()

In [ ]:
# stop
# stopProcess()
# show process
# !ps -ax | grep python
# proc_queue

### MyTrainer and MyThread

In [11]:
from dopamine.discrete_domains import run_experiment
# MyRunner for Train
# - report every episode status.
class MyRunner(run_experiment.Runner):
    def __init__(self, base_dir, create_agent_fn):
        '''initialize runner'''
        super(MyRunner, self).__init__(base_dir, create_agent_fn)
        self._load_logger()
    def _run_one_episode(self):
        '''override to post episode status'''
        global proc_queue
        episode_length, episode_return = super(MyRunner, self)._run_one_episode()
        data = {'episode':{'length': episode_length, 'return': episode_return }}
        #proc_queue.put('! epsode[len,ret] = {},{}'.format(episode_length, episode_return))
        proc_queue.put(data)
        return episode_length, episode_return
    def _load_logger(self):
        '''load logger to save into file'''
        import logging, os
        # get TF logger
        log = logging.getLogger('tensorflow')
        log.setLevel(logging.DEBUG)        
        # create file handler which logs even debug messages
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fh = logging.FileHandler(os.path.join(DQN_PATH, 'tensorflow.log'))
        fh.setLevel(logging.INFO)
        fh.setFormatter(formatter)
        log.addHandler(fh)


#! start runner
def startMyRunner(name = 'train', Q = None):
    global gin_files, gin_bindings, DQN_PATH
    from dopamine.discrete_domains import run_experiment
    Q.put('! start: my-runner') if Q else None
    tf.logging.set_verbosity(tf.logging.INFO)
    run_experiment.load_gin_configs(gin_files, gin_bindings)
    runner = MyRunner(DQN_PATH, run_experiment.create_agent)
    runner.run_experiment()
    Q.put('! P[{}].stop()'.format(name)) if Q else None
#! start process of runner
startProcessTrain(target = startMyRunner)

<Process(Process-1, started)>

! create-retro-game: BubbleBobble-Nes/Level01
! RetroPreprocessing: wall_offset=0, step_penality=-0.001
INFO:tensorflow:Creating ImplicitQuantileAgent agent with the following parameters:
INFO:tensorflow:	 gamma: 0.990000
INFO:tensorflow:	 update_horizon: 3.000000
INFO:tensorflow:	 min_replay_history: 20000
INFO:tensorflow:	 update_period: 4
INFO:tensorflow:	 target_update_period: 8000
INFO:tensorflow:	 epsilon_train: 0.010000
INFO:tensorflow:	 epsilon_eval: 0.001000
INFO:tensorflow:	 epsilon_decay_period: 250000
INFO:tensorflow:	 tf_device: /gpu:0
INFO:tensorflow:	 use_staging: True
INFO:tensorflow:	 optimizer: <tensorflow.python.training.adam.AdamOptimizer object at 0x7fa637ee8198>
INFO:tensorflow:	 max_tf_checkpoints_to_keep: 4
INFO:tensorflow:Creating a OutOfGraphPrioritizedReplayBuffer replay memory with the following parameters:
INFO:tensorflow:	 observation_shape: (84, 84)
INFO:tensorflow:	 observation_dtype: <class 'numpy.uint8'>
INFO:tensorflow:	 terminal_dtype: <class 'numpy.

#### Train results : ~02/06/2020 w/ WallFilter

```pre
INFO:tensorflow:Starting iteration 6
INFO:tensorflow:Average undiscounted return per training episode: 12.42
INFO:tensorflow:Average training steps per second: 77.21
INFO:tensorflow:Starting iteration 7
INFO:tensorflow:Average undiscounted return per training episode: 28.87
INFO:tensorflow:Average training steps per second: 78.09
INFO:tensorflow:Starting iteration 19
INFO:tensorflow:Average undiscounted return per training episode: 27.84
INFO:tensorflow:Average training steps per second: 78.31
INFO:tensorflow:Starting iteration 20
INFO:tensorflow:Average undiscounted return per training episode: 33.08
INFO:tensorflow:Average training steps per second: 78.5
```

![image.png](attachment:image.png)

In [14]:
# MyThread for status display
class MyTrainStatus(MyThread):
    def __init__(self):
        super().__init__(name='status')
        self.episodes = np.array([[0,0]])
        print('! MyTrainStatus({})'.format(self.name))
    def display(self):
        global canvas, proc_queue, plt
        episodes = []
        # pop all queue...
        while not proc_queue.empty():
            msg = proc_queue.get()
            if msg and 'episode' in msg:
                E = msg['episode']
                episodes.append([E['length'], E['return']])
        # print('>> episodes = {}'.format(episodes))
        # draw plot if len > 0
        if len(episodes) > 0:
            arr = np.array(episodes)
            print('>> arr = {}'.format(arr))
            # draw plot...
            if 1>0:
                self.episodes = np.vstack((self.episodes, arr))
                #print('>> self.episodes = {}'.format(self.episodes))            
                #fig = plt.figure(1)
                #plt.plot(self.episodes)
                fig, ax1 = plt.subplots()
                ax2 = ax1.twinx()
                ax1.plot(self.episodes[:,0], 'g-')
                ax2.plot(self.episodes[:,1], 'b-')
                ax1.set_xlabel('episode count')
                ax1.set_ylabel('length', color='g')
                ax2.set_ylabel('return', color='b')
                drawPlot2Canvas(fig)
                
#! start thread for status
tstatus = MyTrainStatus()

! MyTrainStatus(status)


In [ ]:
episode_length, episode_return = 1,3
msg = {'episode':{'length': episode_length, 'return': episode_return }}
proc_queue.put(msg)
print('> msg.org = {}'.format(msg))

In [ ]:
# stop - thread of status
tstatus.quit() if tstatus else None

In [15]:
# stop - process of train
stopProcess()

KeyboardInterrupt: 